In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PRETRAINED_MODEL_DIR = 'drive/My Drive/MLLU/finalProject/pretrained_models/base'

In [0]:
DATA_DIR = 'drive/My Drive/MLLU/finalProject/data'
MODEL_DIR = 'drive/My Drive/MLLU/finalProject/model/copa'
PREDICTION_DIR = 'drive/My Drive/MLLU/finalProject/model/copa'

In [4]:
!pip install -qU t5 

     |████████████████████████████████| 143kB 6.4MB/s 
     |████████████████████████████████| 296kB 19.6MB/s 
     |████████████████████████████████| 1.0MB 20.5MB/s 
     |████████████████████████████████| 8.3MB 18.6MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 645kB 30.0MB/s 
     |████████████████████████████████| 3.3MB 45.5MB/s 
     |████████████████████████████████| 421.8MB 26kB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
     |████████████████████████████████| 3.8MB 38.2MB/s 
     |████████████████████████████████| 450kB 31.3MB/s 
     |████████████████████████████████| 3.9MB 36.3MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible

In [5]:
import functools
import t5
import torch
import transformers
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# model = t5.models.HfPyTorchModel("t5-small", "/tmp/hft5/", device)
model = t5.models.HfPyTorchModel("t5-base", MODEL_DIR, device)

In [0]:
import tensorflow_datasets as tfds
sample = True

def dataset_fn(split, shuffle_files=False):
  # train: 400   validation: 100    test: 500
  
  # We only have one file for each split.
  del shuffle_files

  ds = tfds.load(
    "super_glue/copa",
    data_dir=DATA_DIR,
    # Download data locally for preprocessing to avoid using GCS space.
    download_and_prepare_kwargs={"download_dir": "./downloads"})
    #print("A few raw validation examples...")

  if split == 'validation':
    if sample:
      ds = ds['validation'].take(50)
    else:
      ds = ds['validation']
      
  elif split == "train":
    if sample:
      ds = ds['train'].take(200)
    else:
      ds = ds['train']
  # else:
  #   ds = ds['test']

  return ds


In [0]:
def label_preprocessor(ds):
  
  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text
  
  def to_inputs_and_targets(ex):
    return {
        "inputs": "copa choice1: "+ex["choice1"]+"  choice2: "+
                  ex["choice2"]+"  premise: "+ex["premise"]+"  question: "+ex["question"],
        "targets":"i" if ex["label"] == 0 else "eutelelwld",    # change labels
        "idx": ex["idx"]
    }

  return ds.map(to_inputs_and_targets,
               num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [8]:
t5.data.TaskRegistry.remove("copa")
t5.data.TaskRegistry.add(
    "copa",
    # A TfdsTask takes in a TFDS name instead of a tf.data.Dataset function.
    dataset_fn=dataset_fn,
    splits=["train", "validation"],
    sentencepiece_model_path=t5.data.DEFAULT_SPM_PATH,
    text_preprocessor=[label_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.accuracy]
)

# Load and print a few examples.
# check seq_len
import tensorflow.compat.v1 as tf

cola_task = t5.data.TaskRegistry.get("copa")
ds = cola_task.get_dataset(split="train", sequence_length={"inputs": 64, "targets":16})
print("A few preprocessed train examples...")
seq_len_max = 0
input_len_max = 0
for ex in tfds.as_numpy(ds):
  seq_len_max = max(seq_len_max, ex['targets'].shape[0])
  input_len_max = max(input_len_max, ex['inputs'].shape[0])
print('seq_len_max:', seq_len_max)
print('input_len_max:', input_len_max)
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2


A few preprocessed train examples...
seq_len_max: 9
input_len_max: 52
{'inputs_plaintext': b"copa choice1: The ship crashed into the pier.  choice2: The ship's debris sunk in the sea.  premise: The navy bombed the ship.  question: effect", 'inputs': array([ 7326,     9,  1160,   536,    10,    37,  4383, 24679,   139,
           8,     3,  8343,     5,  1160,   357,    10,    37,  4383,
          31,     7, 12544,     3,     7,  6513,    16,     8,  2805,
           5,     3, 17398,    10,    37, 23118,  6417,    15,    26,
           8,  4383,     5,   822,    10,  1504,     1]), 'targets_plaintext': b'eutelelwld', 'targets': array([   3,   15,   76, 1931,   40,  210,   40,   26,    1]), 'idx': 145}
{'inputs_plaintext': b'copa choice1: She opened her parachute.  choice2: She jumped out of the plane.  premise: The skydiver glided safely to the ground.  question: cause', 'inputs': array([ 7326,     9,  1160,   536,    10,   451,  2946,   160,   260,
        1836,  2810,     5,  1160,   

In [0]:
# mrpc_task = t5.data.TaskRegistry.get("copa")
# ds = mrpc_task.get_dataset(split='validation', sequence_length={"inputs": 64, "targets": 4})
# print("A few preprocessed validation examples...")
# max_ = 0
# count = 0
# for ex in tfds.as_numpy(ds.take(20)):
#     print(ex['inputs_plaintext'])
#     count += 1
#     max_=max(max_,len(ex["targets"]))
# print(max_)
# print(count)

In [10]:
# Evaluate the pre-trained checkpoint, before further fine-tuning
model.eval(
    "copa",
    sequence_length={"inputs": 64, "targets": 16},
    batch_size=128,
)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:eval/copa/accuracy at step 0: 0.000


In [11]:
# Run 1000 steps of fine-tuning
model.train(
    mixture_or_task_name="copa",
    steps=2000,
    save_steps=200,
    sequence_length={"inputs": 64, "targets": 16},
    split="train",
    batch_size=32,
    optimizer=functools.partial(transformers.AdamW, lr=1e-4),
)

INFO:absl:Load dataset info from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Reusing dataset super_glue (drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from drive/My Drive/MLLU/finalProject/data/super_glue/copa/1.0.2
INFO:absl:Saving checkpoint for step 0


ValueError: ignored

In [0]:
# Evaluate after fine-tuning
model.eval(
    "copa",
    checkpoint_steps="all",
    sequence_length={"inputs": 64, "targets": 16},
    batch_size=128,
)